# Library importing

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import joblib

# data pre-proccessing

## data reading

In [2]:
final_data = pd.read_csv("/kaggle/input/depi-dataset/data.csv")

In [3]:
categorical_cols = ['store_id','event_name_1','event_name_2','event_type_1','event_type_2',"state_id"]
numeric_cols = ["wm_yr_wk", "wday", "snap","year", "month", "day", "sell_price","lag_1",'price_flag']
categorical_cols += ["item_category", "item_subcategory",'item_number']
categorical_transformer = Pipeline(steps=[
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ("scaler", MinMaxScaler())
])
numeric_transformer = "passthrough"
preprocessor = ColumnTransformer(transformers=[
    ("cat", categorical_transformer, categorical_cols),
    ("num", numeric_transformer, numeric_cols)
])

## Data spliting

In [5]:
final_data.to_csv("/kaggle/working/data.csv",index= False)

In [6]:
preprocessor = joblib.load("/kaggle/input/depi-dataset/pre-proccess.pkl")

In [7]:
joblib.dump(preprocessor,"/kaggle/working/pre-proccess.pkl")

['/kaggle/working/pre-proccess.pkl']